In [1]:
import json
import os
import sys
from datetime import datetime
from multi_agent_system import MultiAgentSystem

def test_multi_agent_system():
    """Comprehensive test suite for the multi-agent system"""
    
    print("🧪 Multi-Agent AI System Test Suite")
    print("=" * 60)
    
    # Initialize system
    api_key = os.getenv("OPENAI_API_KEY", "test-key")
    if api_key == "test-key":
        print("⚠️  Warning: Using test API key. Set OPENAI_API_KEY environment variable for full functionality.")
    
    system = MultiAgentSystem(api_key)
    test_results = []
    
    def log_test(test_name, status, details=""):
        """Log test results"""
        result = {
            "test": test_name,
            "status": status,
            "timestamp": datetime.now().isoformat(),
            "details": details
        }
        test_results.append(result)
        status_icon = "✅" if status == "PASS" else "❌" if status == "FAIL" else "⚠️"
        print(f"{status_icon} {test_name}: {status}")
        if details:
            print(f"   Details: {details}")
    
    # Test 1: JSON Processing
    print("\n📊 Testing JSON Processing:")
    try:
        json_input = {
            "id": "TEST-001",
            "type": "invoice",
            "data": {"amount": 1000, "vendor": "Test Corp"},
            "sender": "test@example.com"
        }
        
        result = system.process_input(json_input, "json_test_1")
        
        if result.format_type.name == "JSON":
            log_test("JSON Format Detection", "PASS")
        else:
            log_test("JSON Format Detection", "FAIL", f"Expected JSON, got {result.format_type.name}")
        
        if result.extracted_data:
            log_test("JSON Data Extraction", "PASS", f"Extracted {len(result.extracted_data)} fields")
        else:
            log_test("JSON Data Extraction", "FAIL", "No data extracted")
            
    except Exception as e:
        log_test("JSON Processing", "FAIL", str(e))
    
    # Test 2: Email Processing
    print("\n📧 Testing Email Processing:")
    try:
        email_input = """From: customer@example.com
Subject: Test Email for Processing

Dear Support,

This is a test email to verify the email processing functionality.
Please treat this as a medium priority request.

Best regards,
Test Customer"""
        
        result = system.process_input(email_input, "email_test_1")
        
        if result.format_type.name == "EMAIL":
            log_test("Email Format Detection", "PASS")
        else:
            log_test("Email Format Detection", "FAIL", f"Expected EMAIL, got {result.format_type.name}")
        
        if result.sender and result.sender != "unknown":
            log_test("Email Sender Extraction", "PASS", f"Sender: {result.sender}")
        else:
            log_test("Email Sender Extraction", "WARN", "Sender not properly extracted")
        
        if result.thread_id:
            log_test("Thread ID Generation", "PASS", f"Thread: {result.thread_id}")
        else:
            log_test("Thread ID Generation", "FAIL", "No thread ID generated")
            
    except Exception as e:
        log_test("Email Processing", "FAIL", str(e))
    
    # Test 3: Malformed JSON Handling
    print("\n🔧 Testing Error Handling:")
    try:
        malformed_json = {
            "id": 123,  # Wrong type
            "data": "test",
            "unexpected_field": "value"
            # Missing required 'type' field
        }
        
        result = system.process_input(malformed_json, "malformed_test")
        
        if result.anomalies:
            log_test("Anomaly Detection", "PASS", f"Found {len(result.anomalies)} anomalies")
        else:
            log_test("Anomaly Detection", "WARN", "No anomalies detected in malformed data")
            
    except Exception as e:
        log_test("Error Handling", "FAIL", str(e))
    
    # Test 4: Memory Storage and Retrieval  
    print("\n💾 Testing Memory System:")
    try:
        # Store a test result
        test_id = "memory_test_001"
        test_input = {"id": test_id, "type": "test", "data": "memory test"}
        result = system.process_input(test_input, test_id)
        
        # Retrieve the result
        retrieved = system.memory.get_result(test_id)
        
        if retrieved and retrieved.source_id == test_id:
            log_test("Memory Storage/Retrieval", "PASS", f"Retrieved result for {test_id}")
        else:
            log_test("Memory Storage/Retrieval", "FAIL", "Failed to retrieve stored result")
            
    except Exception as e:
        log_test("Memory System", "FAIL", str(e))
    
    # Test 5: Thread Traceability
    print("\n🔗 Testing Thread Traceability:")
    try:
        # Create multiple related emails
        base_email = """From: customer@example.com
Subject: Order Issue #12345

First message in thread."""
        
        follow_up = """From: customer@example.com  
Subject: Re: Order Issue #12345

Follow-up message in same thread."""
        
        result1 = system.process_input(base_email, "thread_test_1")
        result2 = system.process_input(follow_up, "thread_test_2")
        
        if result1.thread_id == result2.thread_id:
            log_test("Thread Continuity", "PASS", f"Same thread ID: {result1.thread_id}")
            
            # Test thread history retrieval
            history = system.get_processing_history(thread_id=result1.thread_id)
            if len(history) >= 2:
                log_test("Thread History Retrieval", "PASS", f"Retrieved {len(history)} messages")
            else:
                log_test("Thread History Retrieval", "WARN", f"Expected 2+ messages, got {len(history)}")
        else:
            log_test("Thread Continuity", "FAIL", "Different thread IDs for related emails")
            
    except Exception as e:
        log_test("Thread Traceability", "FAIL", str(e))
    
    # Test 6: Classification Accuracy (if API key is available)
    if api_key != "test-key":
        print("\n🎯 Testing Classification Accuracy:")
        try:
            # Test obvious invoice content
            invoice_content = """INVOICE #INV-2024-001
            
From: billing@company.com
Amount Due: $1,500.00
Due Date: February 15, 2024

Please remit payment promptly."""
            
            result = system.process_input(invoice_content, "classification_test")
            
            if result.intent.name == "INVOICE":
                log_test("Invoice Classification", "PASS")
            else:
                log_test("Invoice Classification", "WARN", f"Classified as {result.intent.name}")
                
            # Test complaint content
            complaint_content = """From: angry@customer.com
Subject: URGENT COMPLAINT

I am extremely unhappy with your service. The product is defective 
and I demand immediate resolution. This is unacceptable!"""
            
            result = system.process_input(complaint_content, "complaint_test")
            
            if result.intent.name == "COMPLAINT":
                log_test("Complaint Classification", "PASS")
            else:
                log_test("Complaint Classification", "WARN", f"Classified as {result.intent.name}")
                
        except Exception as e:
            log_test("Classification Testing", "FAIL", str(e))
    else:
        log_test("Classification Testing", "SKIP", "No API key provided")
    
    # Test Summary
    print("\n" + "=" * 60)
    print("📋 TEST SUMMARY")
    print("=" * 60)
    
    passed = len([r for r in test_results if r["status"] == "PASS"])
    failed = len([r for r in test_results if r["status"] == "FAIL"]) 
    warned = len([r for r in test_results if r["status"] == "WARN"])
    skipped = len([r for r in test_results if r["status"] == "SKIP"])
    total = len(test_results)
    
    print(f"✅ Passed: {passed}")
    print(f"❌ Failed: {failed}")
    print(f"⚠️  Warnings: {warned}")
    print(f"⏭️  Skipped: {skipped}")
    print(f"📊 Total: {total}")
    
    success_rate = (passed / (total - skipped)) * 100 if (total - skipped) > 0 else 0
    print(f"🎯 Success Rate: {success_rate:.1f}%")
    
    # Save detailed results
    with open("test_results.json", "w") as f:
        json.dump({
            "summary": {
                "passed": passed,
                "failed": failed,
                "warned": warned,
                "skipped": skipped,
                "total": total,
                "success_rate": success_rate,
                "timestamp": datetime.now().isoformat()
            },
            "detailed_results": test_results
        }, f, indent=2)
    
    print(f"\n📄 Detailed results saved to test_results.json")
    
    if failed == 0:
        print("\n🎉 All critical tests passed! System is ready for demo.")
        return True
    else:
        print(f"\n⚠️  {failed} test(s) failed. Please review before demo.")
        return False

if __name__ == "__main__":
    success = test_multi_agent_system()
    sys.exit(0 if success else 1)

✅ API Key set!
🚀 System ready!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.406262 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.877370 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:multi_agent_system:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:multi_agent_system:Classified 32d6ae7a: Format=JSON, Intent=GENERAL
INFO:multi_agent_system:Stored result for 32d6ae7a


SUCCESS: JSON - GENERAL
🧪 Testing main system...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.441496 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.836199 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:multi_agent_system:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:multi_agent_system:Classified a537caca: Format=JSON, Intent=GENERAL
INFO:multi_agent_system:Stored result for a537caca


✅ System working: JSON
🚀 SIMPLE INTERFACE LOADED!

📋 Available Functions:
• test_multi_agent_system() - Run all tests
• process_my_json(data) - Test your JSON
• process_my_email('text') - Test your email
• DEMO_DATA - Pre-made test data

✨ Ready to test! Run: test_multi_agent_system()
🤖 MULTI-AGENT SYSTEM TESTER

1️⃣ TESTING JSON PROCESSING...


...

⚠️ Invoice Classification: WARN
   Details: Classified as GENERAL


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.471801 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 0.795704 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
ERROR:multi_agent_system:Error in intent classification: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
INFO:multi_agent_system:Classified complaint_test: Format=EMAIL, Intent=GENERAL
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/complet

⚠️ Complaint Classification: WARN
   Details: Classified as GENERAL

📋 TEST SUMMARY
✅ Passed: 7
❌ Failed: 1
⚠️  Warnings: 2
⏭️  Skipped: 0
📊 Total: 10
🎯 Success Rate: 70.0%

📄 Detailed results saved to test_results.json

⚠️  1 test(s) failed. Please review before demo.


SystemExit: 1

C:\Users\hp\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.responses import HTMLResponse, JSONResponse
from pydantic import BaseModel
import json
import os
import sys
from pathlib import Path
from typing import Optional, Dict, Any

# Add the current directory to Python path to import your multi_agent_system module
current_dir = Path("multi_agent_system.ipynb").parent
sys.path.append(str(current_dir))

# Now import your multi-agent system
try:
    from multi_agent_system import MultiAgentSystem, ProcessingResult
except ImportError as e:
    print(f"Error importing multi_agent_system: {e}")
    print("Make sure multi_agent_system.py is in the same directory as this file")
    sys.exit(1)

app = FastAPI(title="Multi-Agent AI System Demo", version="1.0.0")

# Initialize the multi-agent system
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "9o0dvHP8hByOQIcJqdSTmk5JFR5wksEyfshPyj1M9LjLNQLUpZVFwCN3SwV1NxmlN5lXRbPzRVT3BlbkFJqtB0_oVZMtf3vstRBFfbAFKd6r8fo91wt5abRe1f4FeSsDlmGlIQ0Z70Hl9H3XezAZstQ32BAA")
if OPENAI_API_KEY == "":
    print("⚠️  Warning: Please set your OPENAI_API_KEY environment variable")

system = MultiAgentSystem(OPENAI_API_KEY)

class JSONInput(BaseModel):
    data: Dict[Any, Any]
    source_id: Optional[str] = None

class EmailInput(BaseModel):
    content: str
    source_id: Optional[str] = None

@app.get("/", response_class=HTMLResponse)
async def demo_interface():
    """Demo web interface"""
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Multi-Agent AI System Demo</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 40px; background: #f5f5f5; }
            .container { max-width: 1200px; margin: 0 auto; background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
            .header { text-align: center; color: #333; margin-bottom: 30px; }
            .section { margin: 20px 0; padding: 20px; border: 1px solid #ddd; border-radius: 5px; }
            .upload-area { background: #f9f9f9; padding: 20px; border: 2px dashed #ccc; border-radius: 5px; text-align: center; }
            textarea { width: 100%; height: 150px; margin: 10px 0; padding: 10px; border: 1px solid #ddd; border-radius: 5px; }
            button { background: #007bff; color: white; padding: 10px 20px; border: none; border-radius: 5px; cursor: pointer; margin: 5px; }
            button:hover { background: #0056b3; }
            .result { background: #e9f7ef; padding: 15px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #28a745; }
            .error { background: #f8d7da; padding: 15px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #dc3545; }
            .demo-section { margin: 20px 0; }
            .demo-button { background: #28a745; margin: 5px; }
            .demo-button:hover { background: #1e7e34; }
            .history-section { background: #f8f9fa; padding: 15px; border-radius: 5px; margin: 10px 0; }
            input[type="text"] { width: 300px; padding: 8px; margin: 5px; border: 1px solid #ddd; border-radius: 3px; }
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>🤖 Multi-Agent AI System Demo</h1>
                <p>Upload PDF, JSON, or Email content for intelligent classification and processing</p>
            </div>
            
            <div class="demo-section">
                <h3>🚀 Quick Demo</h3>
                <button class="demo-button" onclick="runJSONDemo()">Test JSON Processing</button>
                <button class="demo-button" onclick="runEmailDemo()">Test Email Processing</button>
                <button class="demo-button" onclick="runComplaintDemo()">Test Complaint Processing</button>
            </div>
            
            <div class="section">
                <h3>📄 PDF Upload</h3>
                <div class="upload-area">
                    <input type="file" id="pdfFile" accept=".pdf" />
                    <button onclick="processPDF()">Process PDF</button>
                </div>
            </div>
            
            <div class="section">
                <h3>📧 Email Content</h3>
                <textarea id="emailContent" placeholder="Paste email content here...
From: sender@example.com
Subject: Your subject here
                
Your email content..."></textarea>
                <button onclick="processEmail()">Process Email</button>
            </div>
            
            <div class="section">
                <h3>🔧 JSON Data</h3>
                <textarea id="jsonContent" placeholder='{"id": "example", "type": "invoice", "data": {...}}'></textarea>
                <button onclick="processJSON()">Process JSON</button>
            </div>
            
            <div class="section history-section">
                <h3>📊 Processing History</h3>
                <div>
                    <input type="text" id="sourceId" placeholder="Source ID" />
                    <button onclick="getHistory()">Get History by Source ID</button>
                </div>
                <div>
                    <input type="text" id="threadId" placeholder="Thread ID" />
                    <button onclick="getThreadHistory()">Get Thread History</button>
                </div>
            </div>
            
            <div id="results"></div>
        </div>
        
        <script>
            function showResult(data, type = 'success') {
                const resultsDiv = document.getElementById('results');
                const className = type === 'error' ? 'error' : 'result';
                resultsDiv.innerHTML = `
                    <div class="${className}">
                        <h4>Processing Result:</h4>
                        <pre>${JSON.stringify(data, null, 2)}</pre>
                    </div>
                ` + resultsDiv.innerHTML;
            }
            
            async function processPDF() {
                const fileInput = document.getElementById('pdfFile');
                if (!fileInput.files[0]) {
                    alert('Please select a PDF file');
                    return;
                }
                
                const formData = new FormData();
                formData.append('file', fileInput.files[0]);
                
                try {
                    const response = await fetch('/process/pdf', {
                        method: 'POST',
                        body: formData
                    });
                    const result = await response.json();
                    showResult(result);
                } catch (error) {
                    showResult({error: error.message}, 'error');
                }
            }
            
            async function processEmail() {
                const content = document.getElementById('emailContent').value;
                if (!content.trim()) {
                    alert('Please enter email content');
                    return;
                }
                
                try {
                    const response = await fetch('/process/email', {
                        method: 'POST',
                        headers: {'Content-Type': 'application/json'},
                        body: JSON.stringify({content: content})
                    });
                    const result = await response.json();
                    showResult(result);
                } catch (error) {
                    showResult({error: error.message}, 'error');
                }
            }
            
            async function processJSON() {
                const content = document.getElementById('jsonContent').value;
                if (!content.trim()) {
                    alert('Please enter JSON data');
                    return;
                }
                
                try {
                    const jsonData = JSON.parse(content);
                    const response = await fetch('/process/json', {
                        method: 'POST',
                        headers: {'Content-Type': 'application/json'},
                        body: JSON.stringify({data: jsonData})
                    });
                    const result = await response.json();
                    showResult(result);
                } catch (error) {
                    showResult({error: error.message}, 'error');
                }
            }
            
            async function getHistory() {
                const sourceId = document.getElementById('sourceId').value;
                if (!sourceId.trim()) {
                    alert('Please enter a source ID');
                    return;
                }
                
                try {
                    const response = await fetch(`/history/${sourceId}`);
                    const result = await response.json();
                    showResult(result);
                } catch (error) {
                    showResult({error: error.message}, 'error');
                }
            }
            
            async function getThreadHistory() {
                const threadId = document.getElementById('threadId').value;
                if (!threadId.trim()) {
                    alert('Please enter a thread ID');
                    return;
                }
                
                try {
                    const response = await fetch(`/thread/${threadId}`);
                    const result = await response.json();
                    showResult(result);
                } catch (error) {
                    showResult({error: error.message}, 'error');
                }
            }
            
            // Demo functions
            async function runJSONDemo() {
                const demoJSON = {
                    "id": "INV-2024-001",
                    "type": "invoice",
                    "vendor": "TechCorp Solutions",
                    "amount": 2500.00,
                    "due_date": "2024-02-15",
                    "items": [
                        {"description": "Software License", "quantity": 5, "price": 500}
                    ],
                    "sender": "billing@techcorp.com"
                };
                
                document.getElementById('jsonContent').value = JSON.stringify(demoJSON, null, 2);
                await processJSON();
            }
            
            async function runEmailDemo() {
                const demoEmail = `From: procurement@company.com
Subject: RFQ - Office Supplies Q1 2024

Dear Vendor,

We are requesting quotations for the following office supplies for Q1 2024:

1. A4 Paper - 100 reams
2. Printer Cartridges - HP LaserJet series
3. Office Chairs - Ergonomic, 25 units
4. Desk Organizers - 50 units

Please provide your best pricing and delivery timeline.

Deadline for submission: January 30, 2024

Best regards,
Sarah Johnson
Procurement Manager`;
                
                document.getElementById('emailContent').value = demoEmail;
                await processEmail();
            }
            
            async function runComplaintDemo() {
                const complaintEmail = `From: angry.customer@email.com
Subject: URGENT - Defective Product Complaint

Dear Customer Service,

I am extremely disappointed with my recent purchase (Order #12345). 
The product arrived damaged and completely unusable. This is the third time 
I've had issues with your company's products.

I demand immediate replacement and compensation for the inconvenience.
This is affecting my business operations and I need resolution TODAY.

If this is not resolved within 24 hours, I will be escalating to management
and considering legal action.

Furious Customer
John Smith`;
                
                document.getElementById('emailContent').value = complaintEmail;
                await processEmail();
            }
        </script>
    </body>
    </html>
    """
    return html_content

@app.post("/process/json")
async def process_json_endpoint(input_data: JSONInput):
    """Process JSON input"""
    try:
        result = system.process_input(input_data.data, input_data.source_id)
        return {
            "status": "success",
            "source_id": result.source_id,
            "format": result.format_type.value,
            "intent": result.intent.value,
            "sender": result.sender,
            "timestamp": result.timestamp.isoformat(),
            "extracted_data": result.extracted_data,
            "anomalies": result.anomalies,
            "thread_id": result.thread_id
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/process/email")
async def process_email_endpoint(input_data: EmailInput):
    """Process email input"""
    try:
        result = system.process_input(input_data.content, input_data.source_id)
        return {
            "status": "success",
            "source_id": result.source_id,
            "format": result.format_type.value,
            "intent": result.intent.value,
            "sender": result.sender,
            "timestamp": result.timestamp.isoformat(),
            "extracted_data": result.extracted_data,
            "anomalies": result.anomalies,
            "thread_id": result.thread_id
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/process/pdf")
async def process_pdf_endpoint(file: UploadFile = File(...)):
    """Process PDF input"""
    try:
        if not file.filename.endswith('.pdf'):
            raise HTTPException(status_code=400, detail="File must be a PDF")
        
        content = await file.read()
        result = system.process_input(content, file.filename)
        
        return {
            "status": "success",
            "source_id": result.source_id,
            "format": result.format_type.value,
            "intent": result.intent.value,
            "sender": result.sender,
            "timestamp": result.timestamp.isoformat(),
            "extracted_data": result.extracted_data,
            "anomalies": result.anomalies,
            "thread_id": result.thread_id
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/history/{source_id}")
async def get_processing_history(source_id: str):
    """Get processing history for a source ID"""
    try:
        history = system.get_processing_history(source_id=source_id)
        if not history:
            raise HTTPException(status_code=404, detail="No history found for source ID")
        
        return {
            "status": "success",
            "history": [
                {
                    "source_id": result.source_id,
                    "format": result.format_type.value,
                    "intent": result.intent.value,
                    "sender": result.sender,
                    "timestamp": result.timestamp.isoformat(),
                    "extracted_data": result.extracted_data,
                    "anomalies": result.anomalies,
                    "thread_id": result.thread_id
                } for result in history
            ]
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/thread/{thread_id}")
async def get_thread_history(thread_id: str):
    """Get all processing results for a thread ID"""
    try:
        history = system.get_processing_history(thread_id=thread_id)
        if not history:
            raise HTTPException(status_code=404, detail="No thread history found")
        
        return {
            "status": "success",
            "thread_id": thread_id,
            "message_count": len(history),
            "history": [
                {
                    "source_id": result.source_id,
                    "format": result.format_type.value,
                    "intent": result.intent.value,
                    "sender": result.sender,
                    "timestamp": result.timestamp.isoformat(),
                    "extracted_data": result.extracted_data,
                    "anomalies": result.anomalies
                } for result in history
            ]
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {"status": "healthy", "message": "Multi-Agent AI System is running"}

@app.get("/stats")
async def get_system_stats():
    """Get system processing statistics"""
    try:
        # You can extend this to get more detailed stats from your database
        return {
            "status": "success",
            "message": "System statistics endpoint - extend as needed",
            "endpoints": {
                "process_json": "/process/json",
                "process_email": "/process/email", 
                "process_pdf": "/process/pdf",
                "get_history": "/history/{source_id}",
                "get_thread": "/thread/{thread_id}",
                "health_check": "/health"
            }
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
        
import nest_asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    import uvicorn
    print("🚀 Starting Multi-Agent AI System Demo Server...")
    print("📍 Visit http://localhost:8000 for the demo interface")
    print("📋 API Documentation: http://localhost:8000/docs")
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [7044]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


🚀 Starting Multi-Agent AI System Demo Server...
📍 Visit http://localhost:8000 for the demo interface
📋 API Documentation: http://localhost:8000/docs


SystemExit: 1

C:\Users\hp\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
# enhancement_module.py

import os
import time
import uuid
import asyncio
import redis
import openai
import pdfplumber
import pytesseract
from PIL import Image
from jsonschema import validate
from fastapi import Request, BackgroundTasks, Depends
from fastapi.security import OAuth2PasswordBearer
from email import message_from_bytes
from tenacity import retry, stop_after_attempt, wait_fixed
from dotenv import load_dotenv

# -----------------------------
# Load API Key from .env file
# -----------------------------
load_dotenv()
API_KEY = os.getenv("MY_API_KEY")


# -----------------------------
# LLM Query with Retry + Fallback
# -----------------------------
def query_llm(prompt):
    retries = 3
    for attempt in range(retries):
        try:
            return openai.ChatCompletion.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                api_key=API_KEY
            )
        except openai.error.RateLimitError:
            time.sleep(2 ** attempt)  # Exponential backoff
        except Exception:
            if attempt == retries - 1:
                return use_fallback_model(prompt)  # Fallback to local LLM


def use_fallback_model(prompt):
    return {"response": f"[Fallback] Processed: {prompt}"}


# -----------------------------
# PDF Text Extraction
# -----------------------------
def extract_pdf_text(file_path):
    """Extract text from text-based PDF using pdfplumber"""
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text


def extract_image_text(image_path):
    """Extract text from image using Tesseract OCR"""
    image = Image.open(image_path)
    return pytesseract.image_to_string(image)


# -----------------------------
# Parse MIME Emails (with Attachments)
# -----------------------------
def parse_email(raw_bytes):
    msg = message_from_bytes(raw_bytes)
    subject = msg["subject"]
    body = ""

    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain":
                body += part.get_payload(decode=True).decode()
    else:
        body = msg.get_payload(decode=True).decode()

    return {"subject": subject, "body": body}


# -----------------------------
# Thread Management via Header
# -----------------------------
def get_thread_id(request: Request):
    return request.headers.get("Thread-ID", str(uuid.uuid4()))


def store_conversation(thread_id, message_data):
    # Stub: Store in Redis or DB if needed
    print(f"[Thread {thread_id}] Saved message: {message_data}")


# -----------------------------
# Retry Mechanism with Tenacity
# -----------------------------
@retry(stop=stop_after_attempt(3), wait=wait_fixed(2))
def process_data():
    # Replace this with retryable logic
    print("Processing data...")
    raise Exception("Simulated failure")  # For testing


# -----------------------------
# OAuth2 / JWT Auth (FastAPI)
# -----------------------------
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

def get_current_user(token: str = Depends(oauth2_scheme)):
    # Stub: Replace with JWT decode & role logic
    return {"username": "admin", "role": "superuser"}


# -----------------------------
# Redis for Scalability/Storage
# -----------------------------
r = redis.Redis(host='localhost', port=6379)

def store_message(key, value):
    r.set(key, value)


# -----------------------------
# Async Background Task
# -----------------------------
async def background_task(data):
    await asyncio.sleep(1)  # Simulate delay
    print("Background task complete:", data)


# -----------------------------
# JSON Schema Validation
# -----------------------------
def validate_input(data):
    schema = {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "price": {"type": "number"},
            "is_offer": {"type": "boolean"}
        },
        "required": ["name", "price"]
    }
    validate(instance=data, schema=schema)
